In [1]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook creates the reporting units dataset. It does not, however, summarize data into the reporting units.

The geoprocessing workflow for creating these reporting units is quite complex, but the overall goal is to have a more or less uniform set of polygons that maintain parcel boundary lines, while combining small parcels, and subdividing large parcels, while also paying attention to the WUI interface and intermix zones.

There is also an outputted APN table which can be joined to these reporting units to determine which APNs correspond to which reporting unit. 

**Input Data Layers:**
- Expanded WUI/AOI (created in `1_Expanded_WUI.ipynb`)
- [SBC Parcels](https://databasin.org/datasets/008da0ba3dc14be18aa3bcfd90dc9615/)
- [SBC Boundary](https://databasin.org/datasets/1cdc07adea7d4dee9cc1f07ab44cfef3/)
- [WUI Intermix & Interface](https://databasin.org/datasets/218e54cf56f94342a46fefcf05c64f9a/)

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

**Import starting data layers and create WUI merge**

In [3]:
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')

in_data = ["SBC_Parcels", "SBC_Boundary", "WUI_Intermix", "WUI_Interface"]

for sf in in_data:
    path = os.path.join(THIS_FOLDER, "in_data/{}.shp".format(sf))
    arcpy.CopyFeatures_management(path, sf)
    arcpy.Project_management(sf, "{}_rpj".format(sf), crs)   

arcpy.Merge_management(["WUI_Interface_rpj", "WUI_Intermix_rpj"], "WUI_Merge")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\WUI_Merge'>

**Create the Roads and Gaps layer**